# Sesión 1. Análisis textual en Python

En esta sesión se pretende trabajar con algunos de los conceptos básicos de Python para el procesamiento de texto.


##Apartado 1.0

Descargamos primero el dataset "datasetEspañol.csv" con el que vamos a trabajar.

In [3]:
!wget http://valencia.inf.um.es/valencia-tgine/datasetEspañol.csv

--2024-10-13 09:27:42--  http://valencia.inf.um.es/valencia-tgine/datasetEspa%C3%B1ol.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1953117 (1.9M) [text/csv]
Saving to: ‘datasetEspañol.csv.1’

datasetEspañol.csv. 100%[===================>]   1.86M  2.32MB/s    in 0.8s    

2024-10-13 09:27:43 (2.32 MB/s) - ‘datasetEspañol.csv.1’ saved [1953117/1953117]



## Apartado 1.1
Para ello cargaremos primero el dataset en CSV proporcionado "datasetEspañol.csv" usando la librería **pandas**

Mostraremos también las primeras líneas del CSV cargado

In [4]:
import pandas as pd
import csv



## Apartado 1.2

Seleccionamos únicamente las 200 primeras filas y las columnas 'twitter_id', 'twitter_created_at', 'tweet', 'user' y 'label' y guardamos de nuevo el CSV en el fichero "datasetEspañolReducido.csv".

A partir de ahora trabajaremos con este dataset reducido.

In [5]:
data = pd.read_csv("datasetEspañol.csv", encoding="UTF-8")
columns = ["twitter_id", "twitter_created_at", "tweet", "user", "label"]
data_filtered = data[columns][:200]
data_filtered.to_csv("datasetEspañolfiltrado.csv", encoding="UTF-8", index=False)

## Apartado 1.3

Ahora trabajaremos detectando de manera sencilla algunas expresiones regulares usando la librería **re**.

Para ello seleccionaremos los **hashtags** y **menciones** de los tuits.

Una expresión regular para detectar los hashtags podría ser la siguiente:
\#[A-Za-záéíóúÁÉÍÓÚÜüÑñ0-9\_\-]+

Además, crearemos una nueva columna 'tweet_clean' que no contenga los hashtags ni menciones.

- Usaremos la función "apply" y "lambda" de Pandas.
- Para detectar si la expresión regular existe en un determinado String usaremos la función re.sub()

In [6]:
import re # Importamos la librería para hacer expresiones regulares
data = pd.read_csv("datasetEspañolfiltrado.csv", encoding="UTF-8")

hashtag_reg = "#[A-Za-záéíóúÁÉÍÓÚÜüÑñ0-9_-]+"
mention_reg = "@[A-Za-záéíóúÁÉÍÓÚÜüÑñ0-9_-]+"

data["tweet_clean"] = data["tweet"].apply(lambda x : re.sub(hashtag_reg, "", x))
data["tweet_clean"] = data["tweet_clean"].apply(lambda x : re.sub(mention_reg, "", x))
data.head()

,twitter_id,twitter_created_at,tweet,user,label,tweet_clean
0,1262775925831340033,2020-05-19 20:03:36,Hoy merendola deliciosa! Latte Macchiato Caram...,Lorenhia,positive,Hoy merendola deliciosa! Latte Macchiato Caram...
1,1238776542270029824,2020-03-14 11:38:37,"Muchos ánimos a todos los compañeros, profesio...",VacunaJesusRuiz,positive,"Muchos ánimos a todos los compañeros, profesio..."
2,1238774281775067136,2020-03-14 11:29:38,Hay TANTAS cosas que se pueden hacer en casa: ...,jbautyoficial,positive,Hay TANTAS cosas que se pueden hacer en casa: ...
3,1238811338484469763,2020-03-14 13:56:53,#GabineteDeCrisisUtil #16 Escucha música! la q...,ton1pons,positive,"Escucha música! la que te gusta, pero tambié..."
4,1238917460625166336,2020-03-14 20:58:35,Increible el festival de musica gratuito que h...,Alexiat84,positive,Increible el festival de musica gratuito que h...


## Apartado 1.4

Una vez detectadas las expresiones regulares, procederemos a crear dos nuevas columnas con los **hashtags** y **menciones** respectivamente.

- Podemos usar la función re.findall()

In [7]:
data["hashtags"] = data["tweet"].apply(lambda x: re.findall(hashtag_reg, x))
data["mentions"] = data["tweet"].apply(lambda x: re.findall(mention_reg, x))
data.head()

,twitter_id,twitter_created_at,tweet,user,label,tweet_clean,hashtags,mentions
0,1262775925831340033,2020-05-19 20:03:36,Hoy merendola deliciosa! Latte Macchiato Caram...,Lorenhia,positive,Hoy merendola deliciosa! Latte Macchiato Caram...,"[#yomequedoencasa, #todovaasalirbien, #undiame...",[]
1,1238776542270029824,2020-03-14 11:38:37,"Muchos ánimos a todos los compañeros, profesio...",VacunaJesusRuiz,positive,"Muchos ánimos a todos los compañeros, profesio...","[#CoronavirusESP, #YoMeQuedoEnCasa, #vacunas]",[]
2,1238774281775067136,2020-03-14 11:29:38,Hay TANTAS cosas que se pueden hacer en casa: ...,jbautyoficial,positive,Hay TANTAS cosas que se pueden hacer en casa: ...,"[#YoMeQuedoEnCasa, #quedateEnTuCasa]",[]
3,1238811338484469763,2020-03-14 13:56:53,#GabineteDeCrisisUtil #16 Escucha música! la q...,ton1pons,positive,"Escucha música! la que te gusta, pero tambié...","[#GabineteDeCrisisUtil, #16, #YoMeQuedoEnCasa,...",[]
4,1238917460625166336,2020-03-14 20:58:35,Increible el festival de musica gratuito que h...,Alexiat84,positive,Increible el festival de musica gratuito que h...,[#YoMeQuedoEnCasa],[@NilMoliner]


## Apartado 1.5

Sobre esa nueva columna 'tweet_clean' quitaremos los símbolos de puntuación haciendo uso de la librería **string**

Podemos usar la siguiente función

```
#defining the function to remove punctuation
import string

spanish_punctuation = string.punctuation+'¿'+'¡'
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in spanish_punctuation])
    return punctuationfree

```


In [8]:
# defining the function to remove punctuation
import string

spanish_punctuation = string.punctuation+'¿'+'¡'
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in spanish_punctuation])
    return punctuationfree

data["tweet_clean"] = data["tweet_clean"].apply(lambda x : remove_punctuation(x))
data.head()

,twitter_id,twitter_created_at,tweet,user,label,tweet_clean,hashtags,mentions
0,1262775925831340033,2020-05-19 20:03:36,Hoy merendola deliciosa! Latte Macchiato Caram...,Lorenhia,positive,Hoy merendola deliciosa Latte Macchiato Carame...,"[#yomequedoencasa, #todovaasalirbien, #undiame...",[]
1,1238776542270029824,2020-03-14 11:38:37,"Muchos ánimos a todos los compañeros, profesio...",VacunaJesusRuiz,positive,Muchos ánimos a todos los compañeros profesion...,"[#CoronavirusESP, #YoMeQuedoEnCasa, #vacunas]",[]
2,1238774281775067136,2020-03-14 11:29:38,Hay TANTAS cosas que se pueden hacer en casa: ...,jbautyoficial,positive,Hay TANTAS cosas que se pueden hacer en casa v...,"[#YoMeQuedoEnCasa, #quedateEnTuCasa]",[]
3,1238811338484469763,2020-03-14 13:56:53,#GabineteDeCrisisUtil #16 Escucha música! la q...,ton1pons,positive,Escucha música la que te gusta pero también ...,"[#GabineteDeCrisisUtil, #16, #YoMeQuedoEnCasa,...",[]
4,1238917460625166336,2020-03-14 20:58:35,Increible el festival de musica gratuito que h...,Alexiat84,positive,Increible el festival de musica gratuito que h...,[#YoMeQuedoEnCasa],[@NilMoliner]


## Apartado 1.6

Cambiamos el texto de la columna 'tweet_clean' y lo podemos todo en *lowercase*.

Para eso utilizamos la función lower() del objeto string

In [9]:
# La función lower haze minúscula y la función strip convierte varios espacio "    " en uno " ".
data["tweet_clean"] = data["tweet_clean"].apply(lambda x : x.lower().strip())
data[["tweet", "tweet_clean"]].head()

,tweet,tweet_clean
0,Hoy merendola deliciosa! Latte Macchiato Caram...,hoy merendola deliciosa latte macchiato carame...
1,"Muchos ánimos a todos los compañeros, profesio...",muchos ánimos a todos los compañeros profesion...
2,Hay TANTAS cosas que se pueden hacer en casa: ...,hay tantas cosas que se pueden hacer en casa v...
3,#GabineteDeCrisisUtil #16 Escucha música! la q...,escucha música la que te gusta pero también la...
4,Increible el festival de musica gratuito que h...,increible el festival de musica gratuito que h...


## Apartado 1.7

Aplicamos un tokenizer sencillo y guardamos todos los tokens de los tuits limpios en otra columna 'tweet_clean_tokens' usando la siguiente función sencilla de Tokenizer.

```
#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens
```



In [10]:
# defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens

data["tokens"] = data["tweet_clean"].apply(lambda x : tokenization(x))
data[["tweet_clean", "tokens"]].head()

,tweet_clean,tokens
0,hoy merendola deliciosa latte macchiato carame...,"[hoy, merendola, deliciosa, latte, macchiato, ..."
1,muchos ánimos a todos los compañeros profesion...,"[muchos, ánimos, a, todos, los, compañeros, pr..."
2,hay tantas cosas que se pueden hacer en casa v...,"[hay, tantas, cosas, que, se, pueden, hacer, e..."
3,escucha música la que te gusta pero también la...,"[escucha, música, la, que, te, gusta, pero, ta..."
4,increible el festival de musica gratuito que h...,"[increible, el, festival, de, musica, gratuito..."


## Apartado 1.8

**NLTK** es una librería con distintas herramientas para el PLN. La vamos a utilizar para descargar las stopwords en español y para usar su stemmer.

El siguiente paso sería eliminar las stopwords de los tokens usando la librería **NLTK**. Ver función siguiente.



```
import nltk
#Stop words present in the library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('spanish')


#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
    
```



In [11]:
import nltk
# Stop words present in the library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('spanish')

# defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

data["tokens"] = data["tokens"].apply(lambda x : remove_stopwords(x))
data["tokens"].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,tokens
0,"[hoy, merendola, deliciosa, latte, macchiato, ..."
1,"[ánimos, compañeros, profesionales, sanitarios..."
2,"[tantas, cosas, pueden, hacer, casa, ver, cine..."
3,"[escucha, música, gusta, hace, años, escuchaba..."
4,"[increible, festival, musica, gratuito, organi..."


## Apartado 1.9

Por último usando el SnowballStemmer de NLTK obtenemos los stems de cada una de los tokens sin las stopwords y lo guardamos en otra columna 'tweet_clean_stemmed_tokens'



```
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

#defining a function for stemming
def stemming(text):
  stem_text = [stemmer.stem(word) for word in text]
  return stem_text
  ```



In [12]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

# defining a function for stemming
def stemming(text):
  stem_text = [stemmer.stem(word) for word in text]
  return stem_text

data['tweet_clean_stemmed_tokens'] = data["tokens"].apply(lambda x : stemming(x))
data[["tokens", 'tweet_clean_stemmed_tokens']].head()

,tokens,tweet_clean_stemmed_tokens
0,"[hoy, merendola, deliciosa, latte, macchiato, ...","[hoy, merendol, delici, latt, macchiat, carame..."
1,"[ánimos, compañeros, profesionales, sanitarios...","[anim, compañer, profesional, sanitari, hoy, t..."
2,"[tantas, cosas, pueden, hacer, casa, ver, cine...","[tant, cos, pued, hac, cas, ver, cin, seri, le..."
3,"[escucha, música, gusta, hace, años, escuchaba...","[escuch, music, gust, hac, años, escuch, telet..."
4,"[increible, festival, musica, gratuito, organi...","[increibl, festival, music, gratuit, organiz, ..."


##Apartado 1.10  Simple corrección ortográfica (Resuelto)
Muchos textos tienen errores léxicos y hay distintas librerías para la corrección ortográfica a partir de diccionarios. Una de ellas es la librería **pyspellchecker**

Hay otras opciones como hunspell y pyenchant que hacen una corrección léxica basada en diccionarios

In [13]:
# instalamos la libería
!pip3 install pyspellchecker

#importamos la librería
import spellchecker

texto_erróneo = "La asginatura del master haze trabajar y aprehnder procesamiengo de teexto"

# Crea un objeto SpellChecker para el idioma especificado
spell = spellchecker.SpellChecker(language='es')

# Divide el texto en palabras
palabras = texto_erróneo.split()

# Inicializa una lista para las palabras corregidas
palabras_corregidas = []

# Verifica cada palabra en el texto
for palabra in palabras:
# Si la palabra está mal escrita, sugiere correcciones
   correccion = spell.correction(palabra)
   palabras_corregidas.append(correccion)

# Unimos las palabras corregidas para formar el texto corregido
texto_corregido = ' '.join(palabras_corregidas)
print(texto_corregido)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.6 MB/s eta 0:00:00
La asignatura del aster haz trabajar y aprender procesamiento de texto


## Ejercicio a resolver y entregar
Una vez visto el framework stanza en el siguiente Notebook P1.2, crear una columna 'tweet_entities' con las entidades del texto.

Una mejora de este ejercicio es crear una columna para cada tipo de entidad detectada. Esto es necesario para tener la máxima nota en el ejercicio

Debido a que puede tardar bastante tiempo, podéis hacerlo con un subconjunto del dataset de unas 20 líneas.


In [17]:
!pip install stanza
import stanza
stanza.download("es")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 26.1 MB/s eta 0:00:00


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [142]:
data = pd.read_csv("datasetEspañol.csv", encoding="UTF-8")
data = data.iloc[:20]
data.head()

,twitter_id,twitter_created_at,tweet,corpus,user,agreement,votes,score,label,__split
0,1262775925831340033,2020-05-19 20:03:36,Hoy merendola deliciosa! Latte Macchiato Caram...,Estado de alarma nacional (oficial),Lorenhia,100,3,3,positive,val
1,1238776542270029824,2020-03-14 11:38:37,"Muchos ánimos a todos los compañeros, profesio...",Estado de alarma nacional (oficial),VacunaJesusRuiz,100,2,2,positive,train
2,1238774281775067136,2020-03-14 11:29:38,Hay TANTAS cosas que se pueden hacer en casa: ...,Estado de alarma nacional (oficial),jbautyoficial,100,2,2,positive,train
3,1238811338484469763,2020-03-14 13:56:53,#GabineteDeCrisisUtil #16 Escucha música! la q...,Estado de alarma nacional (oficial),ton1pons,100,2,2,positive,train
4,1238917460625166336,2020-03-14 20:58:35,Increible el festival de musica gratuito que h...,Estado de alarma nacional (oficial),Alexiat84,100,2,2,positive,train


In [200]:
from typing import List

# Eliminamos los # y @ de los tweets
hashtag_reg = "#[A-Za-záéíóúÁÉÍÓÚÜüÑñ0-9_-]+"
mention_reg = "@[A-Za-záéíóúÁÉÍÓÚÜüÑñ0-9_-]+"

data["tweet_clean"] = data["tweet"].apply(lambda x : re.sub(hashtag_reg, "", x))
data["tweet_clean"] = data["tweet_clean"].apply(lambda x : re.sub(mention_reg, "", x))


# Creamos el pipeline para detectar entidades
pipelineStanza = stanza.Pipeline(lang="es", processor=("tokenize", "ner"))


# Definimos una función para extraer las entidades de cada tipo en un texto
def extract_entities(text: str) -> List[List[str]]:
  # Aplicamos el pipeline
  stanzaDoc = pipelineStanza(text)

  # Creamos un diccionario vacío de las entidades
  entities = {
        "PER": [],
        "LOC": [],
        "ORG": [],
        "MISC": []
    }

  # Hacemos un bucle para detectar las entidades y clasificarlas según su tipo
  # en el diccionario
  for sentence in stanzaDoc.sentences:
    for entity in sentence.entities:
      entities[f"{entity.type}"].append(entity.text)

  return entities.values()

# Guardamos cada uno de los tres nuevos resultados dentro de 3 nuevas columnas del dataset gracias a la función zip de python
data["entities_PER"], data["entities_LOC"], data["entitites_ORG"], data["entities_MISC"] = zip(*data["tweet_clean"].apply(extract_entities))


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [201]:
data[["tweet_clean", "tweet_entities_PER", "tweet_entities_ORG", "tweet_entities_MISC", "tweet_entities_LOC"]]

,tweet_clean,tweet_entities_PER,tweet_entities_ORG,tweet_entities_MISC,tweet_entities_LOC
0,Hoy merendola deliciosa! Latte Macchiato Caram...,,,,
1,"Muchos ánimos a todos los compañeros, profesio...",,,SARS-COV-2,
2,Hay TANTAS cosas que se pueden hacer en casa: ...,,,,
3,"Escucha música! la que te gusta, pero tambié...",,,,
4,Increible el festival de musica gratuito que h...,Soldadito,,,
5,". con mis dos bichoncitos,me hacen el día?? ...",,,,
6,Gracias espero que hayas tenido buena guardia!...,,,,
7,Segundo día de encierro.? Un vecino se arranca...,Palma,,,
8,Bea me representas! ー19,,,,
9,"En situaciones excepcionales, medidas excepcio...",,,,
